In [2]:
# import libraries
import pandas as pd
import numpy as np

In [3]:
# read file
df = pd.read_excel('../data_and_output/110255_R2_Report.xlsx')

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


***

In [5]:
# main portfolios excluding mains with NO associates
main_df = df[(df['Category'] == 'Main') &
             (df['Portfolio Number'].isin(df['Main Portfolio'].tolist()))]

# associate portfolios
assc_df = df[df['Category'] == 'Associated']

# legacy -02 portfolios (has value in main pf but main pf is closed and not in pf list)
legacy_df = df[(~df['Main Portfolio'].isna()) & 
               (~df['Main Portfolio'].isin(df['Portfolio Number'].tolist()))]

In [6]:
# list of all legacy -02 main portfolios
legacy_list = legacy_df['Portfolio Number'].drop_duplicates().tolist()

legacy_02_list = []
for pf in legacy_list: # loop to pick out the 'main' -02 portfolios only
    if pf[-2:] == '02': legacy_02_list.append(pf) # only portfolios ending with -02
    else: continue

# remove all legacy-02 portfolios from assc_df
assc_df = assc_df.drop(assc_df[assc_df['Portfolio Number'].isin(legacy_02_list)].index)

# add all legacy-02 portfolios to main_df
main_df = pd.concat([main_df, legacy_df[legacy_df['Portfolio Number'].isin(legacy_02_list)]], ignore_index=True)

In [7]:
# in assc_df, replace the main portfolio for all associate portfolios of legacy-02 portfolios
for pf in legacy_df['Main Portfolio'].tolist(): #for each legacy portfolio's -01 portfolio
    
    new_main_list = legacy_df.loc[legacy_df['Main Portfolio']==pf]['Portfolio Number'].tolist()
    
    if new_main_list: # if list not empty = if TRUE = pf is a legacy -02 portfolio
        new_main_pf = new_main_list[0]
        assc_df.loc[assc_df['Main Portfolio'] == pf, 'Main Portfolio'] = new_main_pf # update main portfolio of legacy assc
        main_df.loc[main_df['Main Portfolio'] == pf, 'Main Portfolio'] = new_main_pf # update main portfolio of legacy 02
        main_df.loc[main_df['Main Portfolio'] == pf, 'Category'] = 'Main' # update category of legacy 02
    
    else: continue # if pf is NOT a legacy -02 portfolio

***

In [9]:
# create identifying keys for each party/partyrole
main_df['Key'] = main_df['Party ID'] + main_df['Relationship (Party Role)']
assc_df['Key'] = assc_df['Party ID'] + assc_df['Relationship (Party Role)']

In [10]:
# list of main portfolios
main_list = main_df['Portfolio Number'].drop_duplicates().tolist()

total_main_list = []
total_assc_list = []

for main in main_list: # iterate through main portfolios
    
    # list of all portfolios associated with 'main'
    assc_list = assc_df[assc_df['Main Portfolio'] == main]['Portfolio Number'].drop_duplicates().tolist()
    
    # create set of all parties and respective roles in 'main'
    main_party = main_df[main_df['Portfolio Number'] == main]['Key'].tolist()
    main_party_set = set(main_party)
    
    for assc in assc_list: # iterate through main's associated portfolios
        
        # create set of all parties and respective roles in 'main'
        assc_party = assc_df[assc_df['Portfolio Number'] == assc]['Key'].tolist()
        assc_party_set = set(assc_party)
        
        main_assc = main_party_set.difference(assc_party_set)
        assc_main = assc_party_set.difference(main_party_set)
        
        if  main_assc: # if main_assc is NOT empty = if TRUE
        
            temp_main_dict = {}
            temp_main_dict['Main Portfolio'] = main
            temp_main_dict['Party Present in'] = main
            #temp_main_dict['Portfolio Opening Date'] = df[df['portfolio number']==main]['portfolio opening date']
            temp_main_dict['Party Not Found in'] = assc
            #temp_main_dict['Portfolio Opening Date.2'] = df[df['portfolio number']==assc]['portfolio opening date']
            temp_main_dict['Party'] = main_assc
            
            total_main_list.append(temp_main_dict)
        
        if assc_main:
        
            temp_assc_dict = {}
            temp_assc_dict['Main Portfolio'] = main
            temp_assc_dict['Party Present in'] = assc
            #temp_assc_dict['Portfolio Opening Date'] = df[df['portfolio number']==assc]['portfolio opening date']
            temp_assc_dict['Party Not Found in'] = main
            #temp_assc_dict['Portfolio Opening Date.2'] = df[df['portfolio number']==main]['portfolio opening date']
            temp_assc_dict['Party'] = assc_main
        
            total_assc_list.append(temp_assc_dict)
        
        else: continue

In [11]:
main_to_assc_df = pd.DataFrame(total_main_list)
assc_to_main_df = pd.DataFrame(total_assc_list)
final_df = pd.concat(['main_to_assc_df', assc_to_main_df])

In [ ]:
# wip

In [ ]:
# split party column into something more readable (df.melt)

# add in standard action team,
# sort by main portfolio number
# finally print to 
#final_df = pd.concat([df1, df2])